In [1]:
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
!pip install transformers
!pip install SentencePiece
import torch
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from transformers import XLNetTokenizer
from keras.preprocessing.sequence import pad_sequences
drive.mount('/content/drive')


     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 35.1MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 
     |████████████████████████████████| 1.2MB 8.8MB/s 
Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/colab_data'
def de_emojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
def text_proc(df, text_col='text'):
    df['orig_text'] = df[text_col]
    # Remove twitter handles
    df[text_col] = df[text_col].apply(lambda x: clean_text(x))
    # Remove URLs
    df[text_col] = df[text_col].apply(lambda x:x.replace('<br />', ' '))
    return df[df[text_col]!='']

In [10]:
data = pd.read_csv(os.path.join(path, "covid-19_articles_data.csv"))
data = text_proc(data,'text').dropna(subset=['sentiment'])
data['len'] = data.text.apply(lambda x: len(x.split(' ')))
print(len(data))
data.head(3)

33324


,Unnamed: 0,text,sentiment,orig_text,len
0,0,If you've recently heard from an old friend yo...,1,"If you've recently heard from an old friend, y...",208
1,1,Dr. Hall says it makes sense that we would rea...,1,Dr. Hall says it makes sense that we would rea...,209
2,2,Please if you are going to protest take care ...,1,"""Please, if you are going to protest, take car...",297


In [16]:
max_length = int(data.len.quantile(0.99))
max_length

349

In [20]:
class XLNet_Dataset(Dataset):

    def __init__(self, text, sentiment, tokenizer, max_len):
        self.text = text
        self.sentiment = sentiment
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': tensor([[  183,    44,    26,   189,  1050,  1133,    40,    48,   532,  1233,
            44,    26,    88,    50,  1412,     9,    79,  3056,  1785,  2420,
           349,    43,    26,    23,    50,  3600,    29,   104,    41,   589,
            22, 28895,    33,   538,    63,    55,   497,   474,    22,     9,
           122,    27,    48,    17,  9035,    17,    88,  1838,  7276,   806,
            20,   104,    26,    23,   796,  1166,    25,    24,   162,    29,
            26,    23,   172,   172,  5280,    21,   885,   332,   100,   730,
            80,    26,   189,   545,   566,   134,    42,   919,     9, 11980,
          1666,  4147,    20, 13858,  4627,    38,    18,   315,    20,  3693,
             9,   919,     9,  1666,    51,    72,   481,    33,     4,     3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1